In [2]:
import pandas as pd

In [9]:
df = pd.read_csv("lab 1 - csv.csv", sep=";")

In [12]:
print(df.columns.tolist())
df.head()

['id', 'name', 'price', 'currency', 'created_at']


,id,name,price,currency,created_at
0,SKU-1001,shoes,799,SEK,2024-01-10
1,SKU-1002,pants,520,SEK,2024/02/15
2,SKU-1003,shirts,450,SEK,NaN
3,NaN,jacket,-200,SEK,2024-13-01
4,SKU-1005,NaN,NaN,NaN,


In [19]:
#normalizing/cleaning the columns
df["id"] = df["id"].str.strip().str.upper().str.replace("_", "-").str.replace(" ", "")

df["name"] = df["id"].str.strip().str.title().str.replace(r"\s+", " ", regex=True)

df["price"] = pd.to_numeric(df["price"], errors="coerce") #make sure its a number

df["currency"] = df["currency"].astype(str).str.strip().str.upper()

df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", yearfirst=True)



In [20]:
#Flagging probable issues

df["Missing_currency_flag"] = df["currency"].isna()
df["Zero_price_flag"] = df["price"] == 0
df["Negative_price"] = df["price"] < 0
df["Luxury_product"] = df["price"] > 100000
df["Missing_product_flag"] = df["price"].isna()


In [24]:
reject_condition = (
    df["Negative_price"] |
    df["Missing_product_flag"] |
    df["id"].isna() | 
    df["name"].isna() 

)

df_rejected = df[reject_condition].copy() #för att pandas ska ej modifiera "df"
df_valid = df[~reject_condition].copy()


In [ ]:
#Generating column files

columns = {
    "Snittpris": df_valid["price"].mean(),
    "Pris(median)": df_valid["price"].median(),
    "Antal produkter": len(df_valid),
    "Antal produkter med saknad pris": df["Missing_product_flag"].sum()

}

columns_df = pd.DataFrame([columns])
columns_df.to_csv("analytics_summary.csv", index=False)